## Design:

Find videos with random numbers until list is of size 200.

In [27]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

In [28]:
# Params:

location = '' # 'RU' or '' for default
startDate = 2009
endDate = 2011
videoDuration = 'short'
vidNumber = 100

## Pre Reqs

In [29]:
load_dotenv()
SECRET_KEY = os.getenv('YOUTUBE_API_KEY')

df = pd.DataFrame(columns=[
    'id',
    'datePublished',
    'title',
    'link',
    'description',
    'author',
    'viewCount'
])

## Helpers

In [30]:
import random
def rand_nums() -> str:
    val = ""
    for x in range (4):
        y = random.randint(0,9)
        val += str(y)
    print(val)
    return val

In [31]:
def convert_year_to_rfc3339(year):
    timestamp = pd.Timestamp(year=year, month=pd.Timestamp.today().month, day=pd.Timestamp.today().day, hour=0, minute=0, second=0, tz='UTC')
    rfc3339_str = timestamp.strftime('%Y-%m-%dT%H:%M:%SZ')
    return rfc3339_str


In [32]:
# def start_day(year):
#     timestamp = pd.Timestamp(year=year, month=pd.Timestamp.today().month, day=pd.Timestamp.today().day-1, hour=0, minute=0, second=0, tz='UTC')
#     rfc3339_str = timestamp.strftime('%Y-%m-%dT%H:%M:%SZ')
#     return rfc3339_str

# def end_day(year):
#     timestamp = pd.Timestamp(year=year, month=pd.Timestamp.today().month, day=pd.Timestamp.today().day-1, hour=23, minute=59, second=0, tz='UTC')
#     rfc3339_str = timestamp.strftime('%Y-%m-%dT%H:%M:%SZ')
#     return rfc3339_str

# start_day(2011)
# end_day(2011)

# TODO: Videos from today n years ago
# TODO: Viewer count for video

# maybe location or other interesting things

Front end notes:
if viewed all go to old ones


## Core

In [ ]:

from apiclient.discovery import build

youtube = build('youtube','v3',developerKey = SECRET_KEY)

while (df.shape[0] < vidNumber): #might need adjustments

    num = rand_nums()

    request = youtube.search().list(
        q=f'IMG {num}',
        part='snippet',
        type='video', 
        maxResults='3',
        publishedAfter=f'{convert_year_to_rfc3339(2009)}',
        publishedBefore=f'{convert_year_to_rfc3339(2011)}',
        videoDuration=f'{videoDuration}',
        videoEmbeddable='true'
        )

    res = request.execute()

    for item in res['items']:

        title = item['snippet']['title']
        video_link = 'https://www.youtube.com/watch?v=' + item['id']['videoId']
        video_id = item['id']['videoId']
        description = item['snippet']['description']
        author = item['snippet']['channelTitle']
        date_published = item['snippet']['publishedAt']

        new_row_df = pd.DataFrame([{
            'id': video_id,
            'datePublished': date_published,
            'title': title,
            'link': video_link,
            'description': description,
            'author': author,
            'viewCount' : 0
        }])

        df = pd.concat([df, new_row_df], ignore_index=True)

In [ ]:
df.head(60)

## Postprocess

Remove all videos where title is not in the IMG_XXXX format

In [ ]:
regex_pattern = r"IMG[_ ]\d{4}(?:\.MOV|\.mp4)?" 

filtered_df = df[df['title'].str.match(regex_pattern, na=False)]
filtered_df = filtered_df.reset_index(drop=True)

filtered_df.head(60)

In [ ]:
# Fetch view count
from apiclient.discovery import build

for idx, row in filtered_df.iterrows():

    print(row['id'])

    youtube = build("youtube", "v3", developerKey=SECRET_KEY)
    
    # Query the videos endpoint with the specified part and video ID
    response = youtube.videos().list(
        part="statistics",
        id=row['id']
    ).execute()
    
    # Extract items from the response
    items = response.get("items", [])
    if not items:
        raise ValueError(f"No items found for videoId: {row['id']}")
    
    # Extract the 'statistics' object, then the 'viewCount'
    stats = items[0].get("statistics", {})
    view_count_str = stats.get("viewCount", 0)

    print(view_count_str)

    filtered_df.at[idx, 'viewCount'] = view_count_str


In [ ]:
filtered_df.head(60)

## Daily videos

In [25]:
daily_path = "../DB/videos.json"

filtered_df.to_json(daily_path, orient='records', lines=False, indent=4)

## Archive for research purposes

In [26]:
# might error if this has not been created previously
archive_path = "../DB/old_videos.json"

archive_df = pd.read_json(archive_path, orient='records')

updated_df = pd.concat([archive_df, filtered_df], ignore_index=True)

updated_df.to_json(archive_path, orient='records', lines=False, indent=4)